# Data Cleaning and Preparation

While people often associate data cleaning with Python data libraries like pandas, SQL can actually do quite a bit and rise to the challenge. We will cover a few essential functions for data cleaning including the handling of null values, conditional logic, and string operations. 

For this section, let's practice with the [FAA bird strike dataset](https://wildlife.faa.gov/home) which is a pretty messy dataset with some cleaning opportunities. If you want to learn more about this dataset, I have developed a whole [exploratory data analysis course on Anaconda](https://learning.anaconda.cloud/exploratory-data-analysis-eda-with-python) with it, albeit using pandas. 

Let's import the database I've prepared and converted into SQLite, for records only since the year 2015. 

In [ ]:
import sqlite3
import pandas as pd 

conn = sqlite3.connect('bird_strike.db')

pd.read_sql("SELECT * FROM BIRD_STRIKE_FAA", conn)


Let's talk first about case expressions and then null values. 

## Case Expressions

Let's say I select these fields from the `BIRD_STRIKE_FAA` table. I want to flag any records that involve a Boeing 737, which has several variants. Let's use a wildcard `LIKE` operator to achieve this. 

In [ ]:
sql = """ SELECT 
AIRPORT_ID,
AIRPORT,
AIRCRAFT

FROM BIRD_STRIKE_FAA
WHERE AIRCRAFT LIKE '%737%'
"""

pd.read_sql(sql, conn) 

If we wanted to check what values are being captured with `%737%` we can use a `DISTINCT` operator. 

In [ ]:
sql = """ 
SELECT DISTINCT AIRCRAFT
FROM BIRD_STRIKE_FAA
WHERE AIRCRAFT LIKE '%737%'
"""

pd.read_sql(sql, conn) 

Now I want to compare the Boeing 737 incidents against the Airbus A320 incidents. We can categorize them like this using a `CASE` expression, and use the `ELSE` branch to flag everything else as `OTHER`. 

In [ ]:
sql = """ 
SELECT INDEX_NR, 
AIRCRAFT, 
INCIDENT_DATE,
STATE, 
AIRPORT_ID, 

CASE 
    WHEN AIRCRAFT LIKE '%737%' THEN 'Boeing 737'
    WHEN AIRCRAFT LIKE 'A-320%' THEN 'Airbus A-320'
    ELSE 'Other'
END AIRCRAFT_OF_INTEREST_FLAG 

FROM BIRD_STRIKE_FAA

"""

pd.read_sql(sql, conn) 

To get an idea of how many records have gotten each of these flags, let's use a `COUNT(*)` paired with a `GROUP BY`.

In [ ]:
sql = """ 
SELECT 

CASE 
    WHEN AIRCRAFT LIKE '%737%' THEN 'Boeing 737-related'
    WHEN AIRCRAFT LIKE 'A-320%' THEN 'Airbus A-320-related'
    ELSE 'Other'
END AIRCRAFT_OF_INTEREST_FLAG, 

COUNT(*) AS RECORD_COUNT

FROM BIRD_STRIKE_FAA

GROUP BY AIRCRAFT_OF_INTEREST_FLAG

"""

pd.read_sql(sql, conn) 

The `CASE` expression is useful for all kinds of data transformations and cleanup. Let's talk about null values next. 

## Identifying Null Values

Let's look at a few fields from the table. Notice how the `RUNWAY` and `PHASE_OF_FLIGHT` fields has some `None` values and the `HEIGHT`, `SPEED`, and `DISTANCE` columns have `NaN`. 

In [ ]:
sql = """
SELECT AIRPORT, RUNWAY, PHASE_OF_FLIGHT, HEIGHT, SPEED, DISTANCE 
FROM BIRD_STRIKE_FAA
"""

pd.read_sql(sql, conn)

pandas is actually turning `NULL` values from the SQLite database into `None` and `NaN` values, depending on the datatype of the column (numeric values will be `NaN`). Nulls can be inconvenient if you do not account for them. Aggregate functions like `SUM`, `MIN`, `MAX`, `COUNT` and `AVG` will ignore null values, which we can leverage later. When you write a `WHERE` condition against a field, it will always ignore null values unless you explicitly handle them. Use the `IS NULL` and `IS NOT NULL` operators to qualify/disqualify null values. 




In [ ]:
sql = """
SELECT AIRPORT, RUNWAY, PHASE_OF_FLIGHT, HEIGHT, SPEED, DISTANCE 
FROM BIRD_STRIKE_FAA
WHERE PHASE_OF_FLIGHT IS NULL
"""

pd.read_sql(sql, conn)

For these fields, let's count the number of records and number of non-null values. 

In [ ]:
sql = """
SELECT COUNT(*) AS RECORD_COUNT, 
COUNT(AIRPORT) AS AIRPORT_VALUES, 
COUNT(RUNWAY) AS RUNWAY_VALUES, 
COUNT(PHASE_OF_FLIGHT) AS PHASE_OF_FLIGHT_VALUES, 
COUNT(HEIGHT) AS HEIGHT_VALUES, 
COUNT(SPEED) AS SPEED_VALUES, 
COUNT(DISTANCE) AS DISTANCE_VALUES 
FROM BIRD_STRIKE_FAA
"""

pd.read_sql(sql, conn)

If we wanted to count the number of `NULL` values (rather than non-`NULL` values), we could subtract those field counts from the record counts, or more elegantly `SUM` the number of true values using `IS NULL`. Since `1` will be yielded for true and `0` for false, we can use this to sum the true values. 

In [ ]:
sql = """
SELECT COUNT(*) AS RECORD_COUNT, 
SUM(AIRPORT IS NULL) AS AIRPORT_NULLS, 
SUM(RUNWAY IS NULL) AS RUNWAY_NULLS, 
SUM(PHASE_OF_FLIGHT IS NULL) AS PHASE_OF_FLIGHT_NULLS, 
SUM(HEIGHT IS NULL) AS HEIGHT_NULLS, 
SUM(SPEED IS NULL) AS SPEED_NULLS, 
SUM(DISTANCE IS NULL) AS DISTANCE_NULLS 
FROM BIRD_STRIKE_FAA
"""

pd.read_sql(sql, conn)

We can also ratio the percentage of nulls in each of these fields. Just make sure to convert the operation to floating point values rather than integers, which can be done by multiplying the expression with `1.0`. 

In [ ]:
sql = """
SELECT COUNT(*) AS RECORD_COUNT, 
1.0 * SUM(AIRPORT IS NULL) / COUNT(*) AS AIRPORT_NULL_RATE, 
1.0 * SUM(RUNWAY IS NULL) / COUNT(*)  AS RUNWAY_NULL_RATE, 
1.0 * SUM(PHASE_OF_FLIGHT IS NULL) / COUNT(*) AS PHASE_OF_FLIGHT_NULL_RATE, 
1.0 * SUM(HEIGHT IS NULL) / COUNT(*) AS HEIGHT_NULL_RATE, 
1.0 * SUM(SPEED IS NULL) / COUNT(*) AS SPEED_NULL_RATE, 
1.0 * SUM(DISTANCE IS NULL) / COUNT(*) AS DISTANCE_NULL_RATE 
FROM BIRD_STRIKE_FAA
"""

pd.read_sql(sql, conn)


If we are interested in bird strikes that happened below 500 feet, have we considered the null values and whether we want to include those? This is why documenting what `null` means for a given field is so important. Perhaps the `HEIGHT` was irrelevant, or could not be measured because an instrument was broken. Or perhaps the pilot hastily filed the report and did not care to jot that information down. Regardless, we need to understand why values can be missing and whether they should be included in a given analysis. 

> A common mistake for SQL beginners is they use `= NULL` rather than `IS NULL`. This does not work! Always use the latter. 

If we want to find records where `HEIGHT` is less than 500 feet, but want to include `NULL` values too, then we need to use an `IS NULL` paired with an `OR` to that condition. 

In [ ]:
sql = """
SELECT AIRPORT, RUNWAY, PHASE_OF_FLIGHT, HEIGHT, SPEED, DISTANCE 
FROM BIRD_STRIKE_FAA

WHERE HEIGHT IS NULL OR HEIGHT < 500
"""

pd.read_sql(sql, conn)

We could also use a `CASE` expression to turn `NULL` height values into `0`, but there is something even better: the `COALESCE` function. It will take a possibly null value and swap it with a different value if it's null. 

In [ ]:
sql = """
SELECT AIRPORT, RUNWAY, PHASE_OF_FLIGHT, HEIGHT, SPEED, DISTANCE 
FROM BIRD_STRIKE_FAA

WHERE COALESCE(HEIGHT, 0) < 500
"""

pd.read_sql(sql, conn)

This coalesce achieves the same thing as a `CASE` expression converting null values. 

In [ ]:
sql = """
SELECT AIRPORT, RUNWAY, PHASE_OF_FLIGHT, HEIGHT, SPEED, DISTANCE 
FROM BIRD_STRIKE_FAA

WHERE (CASE WHEN HEIGHT IS NULL THEN 0 ELSE HEIGHT END) < 500
"""

pd.read_sql(sql, conn)

This gives you all the tools you need to filter and handle nulls. Later we will use techniques to impute for missing values with subqueries. There is no once size fits all approach to handling nulls. It will always depend on the task and what is the most appropriate way to handle them. 

## String Operations

A lot of data cleaning will involve string operations. We already saw how to use wildcards with the `LIKE` operator, but there are many functions (and a few operators) targeting strings. 

| Name      | Description                                                                       |
|-----------|-----------------------------------------------------------------------------------|
| LENGTH    | Counts the number of characters in a string.                                      |
| UPPER     | Converts a string to uppercase.                                                   |
| LOWER     | Converts a string to lowercase.                                                   |
| SUBSTR    | Extracts a substring with a predefined length at a specific position.             |
| TRIM      | Removes specified characters (default space) from the start and end of the string. |
| LTRIM     | Removes specified characters (default space) from the start of the string.        |
| RTRIM     | Removes specified characters (default space) from the end of the string. .        |
| REPLACE   | Replaces matching substrings in a string with another substring.                  |
| INSTR     | Returns the position of the first occurrence of the substring, -1 if not found.   |                 
| \|\|      | Concatenates two or more values into a string |
| CONCAT_WS | Concatenates multiple strings into a one string with a separator.                 |
| REGEXP    | Determines if a string matches a regular expression.                              | 


Let's say we wanted to verify if all `AIRPORT_ID` values are four characters. We can use the `LENGTH()` function to do this and sure enough,  there are a handful that are not. 

In [ ]:
sql = """
SELECT * FROM BIRD_STRIKE_FAA
WHERE LENGTH(AIRPORT_ID) != 4
"""

pd.read_sql(sql, conn)

Here is another example where we replace "ARPT" with "AIRPORT" in the `AIRPORT` field. 

In [ ]:
sql = """
SELECT 
AIRPORT,
REPLACE(AIRPORT, 'ARPT','AIRPORT') AS AIRPORT_NEW 
FROM BIRD_STRIKE_FAA
"""

pd.read_sql(sql, conn)

Regular expressions are definitely something you will want to learn when cleaning data, and they are supported in SQL, pandas, Python, and many other platforms. We actually have an [Anaconda course teaching them in Python](https://learning.anaconda.cloud/regular-expressions-in-python). Other SQL platforms will support regular expressions, but SQLite needs it enabled in Python. We can achieve this by running the code below. 

In [ ]:
import re

def regexp(expr, item):
    reg = re.compile(expr)
    return reg.search(item) is not None

conn.create_function("REGEXP", 2, regexp)

Now I can look for both Airbus A-320 and A-321 aircraft using a single regular expression as shown below. 

In [ ]:
sql = """ 
SELECT * FROM BIRD_STRIKE_FAA
WHERE AIRCRAFT REGEXP 'A-32[01]'
"""

pd.read_sql(sql, conn) 

Let's put several of these string operations together for a practical task. Let's take the `INCIDENT_DATE` and `TIME` columns and merge them together into a proper `DATETIME`. We can remove that "0 days " boilerplate on the `TIME` using `SUBSTR`. If the `TIME` value is missing altogether we can use `coalesce()` to replace the null values and make it "00:00:00". Finally we concatenate that with the `INCIDENT_DATE` and cast it all as a `DATETIME`. 

In [ ]:
sql = """
SELECT OPERATOR, 
AIRCRAFT,
AIRPORT,
DATETIME(INCIDENT_DATE || ' ' || COALESCE(SUBSTR(TIME, 7), '00:00:00')) AS INCIDENT_DATETIME 

FROM BIRD_STRIKE_FAA

ORDER BY INCIDENT_DATE DESC
"""

pd.read_sql(sql, conn)

> **Should I write these changes back to the table?** 
> 
> One of the luxuries of SQL is you can easily take a raw data source and make transformations with SQL queries. You might wonder if these changes should be written back into the table. I would only do this in the context that you are doing extract-transform-load (ETL) work and providing a cleaned dataset for others. You can do this easily by calling `CREATE TABLE` with a `SELECT` to make a new table off a `SELECT` query, or using an `INSERT` with a `SELECT` for an existing target table. But you can maintain all your cleanup work in queries that are ran as needed, without having to store the cleaned data back into a table. Consider even sharing the `SELECT` queries themselves, which can simply be stored in a text file, email, or a view. 

## UNION, UNION ALL, and CASE Tricks  

Let's say I am interested in comparing the cost of repairs by year for incidents below 1000 feet and above 1000 feet. Most people would use a `UNION ALL` to append these two queries together. 

In [ ]:
sql = """
SELECT INCIDENT_YEAR, 
'BELOW 500' AS THRESHOLD, 
SUM(COST_REPAIRS) AS TOTAL_REPAIRS
FROM BIRD_STRIKE_FAA
WHERE HEIGHT < 1000 
GROUP BY INCIDENT_YEAR, THRESHOLD

UNION ALL 

SELECT INCIDENT_YEAR, 
'ABOVE 500' AS THRESHOLD, 
SUM(COST_REPAIRS) AS TOTAL_REPAIRS
FROM BIRD_STRIKE_FAA
WHERE HEIGHT >= 1000 
GROUP BY INCIDENT_YEAR, THRESHOLD
"""

pd.read_sql(sql, conn)

This demonstrates the `UNION ALL` which appends the results of both queries together. The `UNION`, which we have not demonstrated, would do the same thing but rid duplicate records. 

I am not a fan of this use case however, as common as it is. The queries are redundant and therefore have to do two scans of the table, which is inefficient. The only difference between the queries is the `WHERE` condition. If we move that `WHERE` condition into a `CASE` expression, we can consolidate into one query. 

In [ ]:
sql = """
SELECT INCIDENT_YEAR, 
CASE WHEN HEIGHT < 1000 THEN 'BELOW 1000' ELSE 'ABOVE 1000' END AS THRESHOLD, 
SUM(COST_REPAIRS) AS TOTAL_REPAIRS
FROM BIRD_STRIKE_FAA
GROUP BY INCIDENT_YEAR, THRESHOLD
"""

pd.read_sql(sql, conn)

We can probably do something even better here. Let's make the `TOTAL_REPAIRS` into two columns, one for the "ABOVE 1000" threshold and another for the "BELOW 1000" threshold. We can do this with two `CASE` expressions inside the `SUM` functions. 

In [ ]:
sql = """
SELECT INCIDENT_YEAR, 
SUM(CASE WHEN HEIGHT < 1000 THEN COST_REPAIRS ELSE NULL END) AS BELOW_1000_COST_REPAIRS, 
SUM(CASE WHEN HEIGHT >= 1000 THEN COST_REPAIRS ELSE NULL END) AS ABOVE_1000_COST_REPAIRS, 
SUM(COST_REPAIRS) AS TOTAL_REPAIRS
FROM BIRD_STRIKE_FAA
GROUP BY INCIDENT_YEAR
"""

pd.read_sql(sql, conn)

Because `NULL` values are ignored by `SUM()` and other aggregate functions, we can use that to conditionally ignore values we do not want to count towards the sum. This effectively allows each `SUM()` function (or any aggregate function) to have different `WHERE` conditions. 

As you can see, the `UNION` and `UNION ALL` are not always a good idea to use and there are often better ways to achieve tasks they are commonly used for, often involving a `CASE` expression. The only use cases they uniquely are qualified for is when you have multiple queries pulling from different tables (not the same one), and transformed into the same structural output to be appended. 

Use the `CASE` expression to enable powerful patterns that few know about! 

## Exercise 


Find the total cost of repairs by `INCIDENT_YEAR` and `INCIDENT_MONTH`, but broken up into two totals: where `SPEED` is less than 200 and `SPEED` is greater than or equal to `200`.  Complete the code below by replacing the question marks `?`. 

In [ ]:
sql = """
SELECT ?, 
?,
? AS BELOW_200_KNOTS_COST_REPAIRS, 
? AS ABOVE_200_KNOTS_COST_REPAIRS, 
SUM(COST_REPAIRS) AS TOTAL_REPAIRS
FROM BIRD_STRIKE_FAA
GROUP BY INCIDENT_YEAR, INCIDENT_MONTH 
"""

pd.read_sql(sql, conn)


### SCROLL DOWN FOR ANSWER
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
v 

In [ ]:
sql = """
SELECT INCIDENT_YEAR, 
INCIDENT_MONTH,
SUM(CASE WHEN SPEED < 200 THEN COST_REPAIRS ELSE NULL END) AS BELOW_200_KNOTS_COST_REPAIRS, 
SUM(CASE WHEN SPEED >= 200 THEN COST_REPAIRS ELSE NULL END) AS ABOVE_200_KNOTS_COST_REPAIRS, 
SUM(COST_REPAIRS) AS TOTAL_REPAIRS
FROM BIRD_STRIKE_FAA
GROUP BY 1, 2
"""

pd.read_sql(sql, conn)